In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from datetime import timedelta
import pandas as pd
import sys
sys.path.append(r"D:\work_dir\stock\src\stockProject")
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import stock

In [196]:
from datetime import timedelta
import numpy as np
import pandas as pd


class Average(object):

    def __init__(self, add_day, line_count, var, col, model='get_ma_A', bf_day=6):
        self.var = var
        self.add_day = add_day
        self.line_count = line_count
        self.bf_day = bf_day
        self.col = col
        self.info_col = ['jx_date', 'jx_names', 'jx_counts', 'var']
        self.model_dict = {i: getattr(self, i) for i in self.__dir__() if i.startswith('get_ma_')}
        self.model = self.model_dict[model]
        self.result = []


    def get_junxian_col(self, data):
        for i in self.col:
            if i not in ['ma5', 'ma10', 'ma20']:
                data[i] = data['close'].rolling(int(i.replace('ma', '')), min_periods=1).mean()
        return data

    def get_slope(self, df):
        y = df.tolist()
        x = list(range(1, len(y) + 1))
        return np.polyfit(x, y, 1)[0]

    def jx_var(self, df):
        mean = df.mean()
        return (df - mean).abs().mean() / mean

    def execute(self, symbol, data):
        data = self.get_junxian_col(data)
        jx_df = data[self.col]
        date_range = jx_df.apply(func=self.jx_var, axis=1)
        date_range = date_range[date_range < self.var].sort_index()
        df_val = pd.DataFrame(date_range,columns=['var']).reset_index().rename(columns={'date':'jx_date'})
        df = self.model(jx_df,symbol)
        result = df_val.merge(df,on=['jx_date'],)
        resp = result.merge(data, left_on=['jx_date', 'symbol'], right_on=['t_date', 'symbol'],suffixes=('','_y'))
        resp.drop(['ma5_y', 'ma10_y', 'ma20_y','ma30_y'],axis=1,inplace=True)
        # for date, var in date_range.iteritems():
        #     print(self.result[-1])
        #     last_date = self.result[-1].jx_date[-1] if self.result else None
        #     print(last_date,date)
        #     if last_date and (date - last_date).days <= 7:
        #         continue
        #     self.result.append(self.model(jx_df, date, var))
        return resp

    def get_ma_A(self, df,symbol):
        '''
        20日线 和 10日线 在 当日前 N 个交易日内 有上升趋势
        且 30 日线 呈上升趋势
        :param df: stock df
        :param st: 交易日开始
        :param en: 当日
        :return: 是否满足均线条件
        '''
        # st = (date + timedelta(days=-self.bf_day)).date().strftime('%Y-%m-%d')
        slope = df.rolling(self.bf_day, min_periods=1).apply(self.get_slope)
        slope['jx_counts'] = (slope > 0).sum(axis=1)
        slope = slope.round(2)

        slope['jx_names'] = slope['ma5'].astype('str') + ',' \
                            + slope['ma10'].astype('str') + ',' \
                            + slope['ma20'].astype('str') + ',' \
                            + slope['ma30'].astype('str')

        slope['symbol'] = symbol
        slope.reset_index(inplace=True,drop=False)
        slope.rename(columns={'date':'jx_date'},inplace=True)
        return slope


    def get_ma_B(self, df, date, var):
        '''
        20日线 和 10日线 在 当日前 N 个交易日内 有上升趋势
        且 30 日线 呈上升趋势
        :param df: stock df
        :param st: 交易日开始
        :param en: 当日
        :return: 是否满足均线条件
        '''
        st = (date + timedelta(days=-self.bf_day)).date().strftime('%Y-%m-%d')
        en = str(date.date())
        buf = df.loc[st:en].diff().fillna(0)
        jx_counts = 0
        jx_names = ""
        jx_jiaocha = pd.DataFrame(df.loc[en].round(2).value_counts())
        jc_counts = len(jx_jiaocha[jx_jiaocha[en] > 1])
        for i in df.columns:
            add = buf[buf[i] > 0].ma5.count()
            if i in ['ma5', 'ma10'] and add < 4:
                jx_counts = 0
                break
            if add >= self.add_day:
                jx_counts += 1
                jx_names += i + ','

        self.info_col = ['jx_date', 'jx_names', 'jx_counts', 'var', 'jc_counts']
        if jx_counts >= self.line_count:
            self.save_cols.append((date, jx_names, jx_counts, var, jc_counts))


In [197]:
df = pd.read_pickle(r"D:\work_dir\stock\src\stockProject\history_data\stock_2020-11-07.pkl")

In [198]:
df.sort_values(['symbol', 'date'], inplace=True)
df.set_index('date', inplace=True, drop=False)
df.date = df.date.astype('datetime64')
df.rename(columns={'date': 't_date'}, inplace=True)
df.index = pd.to_datetime(df.index)

In [199]:
col = ['ma5', 'ma10', 'ma20', 'ma30', ]

In [200]:
df2 = df.query("name=='鲁商发展'").sort_values('date')

In [201]:
%time c = Average(add_day=3, line_count=2, var=0.015, col=col, model='get_ma_A',bf_day=5).execute(df2.symbol[0],df2)

Wall time: 296 ms


c:\users\wuwukai\envs\h1\lib\site-packages\pandas\core\window\rolling.py:1415: RankWarning: Polyfit may be poorly conditioned
  return window_func(values, begin, end, min_periods)
c:\users\wuwukai\envs\h1\lib\site-packages\pandas\core\window\rolling.py:1415: RankWarning: Polyfit may be poorly conditioned
  return window_func(values, begin, end, min_periods)
c:\users\wuwukai\envs\h1\lib\site-packages\pandas\core\window\rolling.py:1415: RankWarning: Polyfit may be poorly conditioned
  return window_func(values, begin, end, min_periods)
c:\users\wuwukai\envs\h1\lib\site-packages\pandas\core\window\rolling.py:1415: RankWarning: Polyfit may be poorly conditioned
  return window_func(values, begin, end, min_periods)


In [202]:
con = stock.pymysql.connect(user='root', port=3306, host='www.chaoyue.red', password='123456', db='bbs_1')

In [203]:
c['sw_class'] = 'get_ma_A_99'
c.rename(columns={"name": 'stock_name', "open": 'open_price', "close": "close_price"}, inplace=True)

In [204]:
stock.write_db(c,con=con,table_name='stock')

In [155]:
c.query("jx_counts>2 and val<=0.015 and ma10>0").reset_index(drop=True)

,jx_date,val,ma5,ma10,ma20,ma30,jx_counts,jx_names,symbol,t_date,open,high,close,low,volume,price_change,p_change,v_ma5,v_ma10,v_ma20,turnover,name,industry,pe
0,2019-11-25,0.013921,0.08,0.01,-0.05,0.07,3,"0.08,0.01,-0.05,0.07",600223,2019-11-25,6.06,6.27,5.71,5.71,777916.62,-0.63,-9.94,827068.02,748564.26,823792.57,7.77,鲁商发展,区域地产,21.44
1,2019-11-26,0.007035,0.04,0.03,-0.07,0.07,3,"0.04,0.03,-0.07,0.07",600223,2019-11-26,5.64,5.85,5.59,5.40,488179.34,-0.12,-2.10,788771.05,742071.56,833938.89,4.88,鲁商发展,区域地产,21.44
2,2019-12-12,0.012122,0.09,0.02,0.02,-0.04,3,"0.09,0.02,0.02,-0.04",600223,2019-12-12,5.89,6.03,5.74,5.70,559094.81,-0.17,-2.88,555142.07,439923.36,580411.83,5.59,鲁商发展,区域地产,21.44
3,2019-12-13,0.011398,0.08,0.03,0.01,-0.02,3,"0.08,0.03,0.01,-0.02",600223,2019-12-13,5.77,5.94,5.74,5.60,440025.88,0.00,0.00,540917.03,449959.77,558279.30,4.40,鲁商发展,区域地产,21.44
4,2019-12-16,0.009211,0.06,0.04,0.01,-0.01,3,"0.06,0.04,0.01,-0.01",600223,2019-12-16,5.72,5.96,5.87,5.68,415541.44,0.13,2.27,505346.21,459194.73,538923.55,4.15,鲁商发展,区域地产,21.44
5,2019-12-17,0.009605,0.04,0.05,0.00,-0.01,3,"0.04,0.05,0.0,-0.01",600223,2019-12-17,5.85,6.12,5.90,5.85,648321.12,0.03,0.51,551328.53,497768.97,537356.40,6.48,鲁商发展,区域地产,21.44
6,2019-12-18,0.012544,0.04,0.06,0.01,-0.01,3,"0.04,0.06,0.01,-0.01",600223,2019-12-18,5.95,6.39,6.24,5.83,839585.31,0.34,5.76,580513.71,544864.02,543725.13,8.39,鲁商发展,区域地产,21.44
7,2020-04-28,0.008747,0.08,0.02,0.00,-0.01,3,"0.08,0.02,0.0,-0.01",600223,2020-04-28,6.99,7.27,7.18,6.60,281612.97,0.09,1.27,292157.88,217208.03,220164.53,2.81,鲁商发展,区域地产,21.44
8,2020-04-29,0.009570,0.08,0.03,0.01,-0.00,3,"0.08,0.03,0.01,-0.0",600223,2020-04-29,7.15,7.25,6.99,6.94,197820.00,-0.19,-2.65,278280.93,221887.03,216383.84,1.98,鲁商发展,区域地产,21.44
9,2020-04-30,0.008758,0.06,0.03,0.01,0.00,4,"0.06,0.03,0.01,0.0",600223,2020-04-30,7.16,7.25,7.15,7.02,241696.11,0.16,2.29,240113.02,233944.24,220209.78,2.41,鲁商发展,区域地产,21.44


In [144]:
c.drop(['ma5_y', 'ma10_y', 'ma20_y','ma30_y'],axis=1)

KeyError: "['ma5_y' 'ma10_y' 'ma20_y' 'ma30_y'] not found in axis"

In [178]:
xx = df2['2019-12-10':'2020-12-24'].ma20

In [181]:
xx

date
2019-12-10     5.629
2019-12-11     5.647
2019-12-12     5.660
2019-12-13     5.664
2019-12-16     5.662
2019-12-17     5.664
2019-12-18     5.696
2019-12-19     5.691
2019-12-20     5.708
2019-12-23     5.759
2019-12-24     5.796
2019-12-25     5.831
2019-12-26     5.896
2019-12-27     5.975
2019-12-30     6.097
2019-12-31     6.254
2020-01-02     6.448
2020-01-03     6.656
2020-01-06     6.905
2020-01-07     7.136
2020-01-08     7.318
2020-01-09     7.502
2020-01-10     7.709
2020-01-13     7.965
2020-01-14     8.194
2020-01-15     8.405
2020-01-16     8.627
2020-01-17     8.855
2020-01-20     9.045
2020-01-21     9.247
2020-01-22     9.492
2020-01-23     9.688
2020-02-03     9.802
2020-02-04     9.883
2020-02-05     9.943
2020-02-06    10.006
2020-02-07    10.005
2020-02-10     9.992
2020-02-11     9.916
2020-02-12     9.856
2020-02-13     9.829
2020-02-14     9.784
2020-02-17     9.742
2020-02-18     9.649
2020-02-19     9.586
2020-02-20     9.529
2020-02-21     9.445
2020-02-

In [179]:
y = xx.tolist()
x = list(range(1,xx.__len__()+1))

In [180]:
np.polyfit(x,y,1)

array([0.02355087, 6.61906563])

In [164]:
df2['close'].rolling(15).max()

date
2019-10-08      NaN
2019-10-09      NaN
2019-10-10      NaN
2019-10-11      NaN
2019-10-14      NaN
2019-10-15      NaN
2019-10-16      NaN
2019-10-17      NaN
2019-10-18      NaN
2019-10-21      NaN
2019-10-22      NaN
2019-10-23      NaN
2019-10-24      NaN
2019-10-25      NaN
2019-10-28     7.94
2019-10-29     7.94
2019-10-30     7.94
2019-10-31     7.94
2019-11-01     7.94
2019-11-04     7.94
2019-11-05     7.94
2019-11-06     7.94
2019-11-07     7.94
2019-11-08     7.94
2019-11-11     7.94
2019-11-12     7.94
2019-11-13     7.94
2019-11-14     7.94
2019-11-15     7.94
2019-11-18     7.15
2019-11-19     6.62
2019-11-20     6.44
2019-11-21     6.44
2019-11-22     6.44
2019-11-25     6.44
2019-11-26     6.44
2019-11-27     6.34
2019-11-28     6.34
2019-11-29     6.34
2019-12-02     6.34
2019-12-03     6.34
2019-12-04     6.34
2019-12-05     6.34
2019-12-06     6.34
2019-12-09     6.34
2019-12-10     6.34
2019-12-11     6.34
2019-12-12     6.34
2019-12-13     5.91
2019-12-16     

In [171]:
def get_slope(df):
    y = df.tolist()
    x = list(range(1,len(y)+1))
    return np.polyfit(x,y,1)[0]

In [226]:
df2.sort_values(['t_date'],ascending=False)

,t_date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover,symbol,name,industry,pe,ma30
date,,,,,,,,,,,,,,,,,,,,
2020-11-06,2020-11-06,12.10,12.10,11.79,11.47,389261.88,-0.41,-3.36,11.376,10.827,10.678,383146.33,299106.54,255103.99,3.89,600223,鲁商发展,区域地产,21.44,10.468667
2020-11-05,2020-11-05,11.77,12.40,12.20,11.60,558382.56,0.65,5.63,11.096,10.660,10.599,358166.68,274665.10,245854.74,5.58,600223,鲁商发展,区域地产,21.44,10.409667
2020-11-04,2020-11-04,10.81,11.90,11.55,10.70,530499.19,0.73,6.75,10.824,10.490,10.472,294447.88,230266.22,223304.64,5.30,600223,鲁商发展,区域地产,21.44,10.335333
2020-11-03,2020-11-03,10.56,10.94,10.82,10.43,225614.73,0.30,2.85,10.580,10.405,10.382,226605.09,193868.55,203846.03,2.25,600223,鲁商发展,区域地产,21.44,10.288000
2020-11-02,2020-11-02,10.50,10.77,10.52,10.36,211973.31,0.13,1.25,10.422,10.389,10.330,220898.55,185756.95,198543.11,2.12,600223,鲁商发展,区域地产,21.44,10.261000
2020-10-30,2020-10-30,10.72,11.04,10.39,10.38,264363.59,-0.45,-4.15,10.278,10.413,10.303,215066.74,191924.76,194663.64,2.64,600223,鲁商发展,区域地产,21.44,10.249000
2020-10-29,2020-10-29,10.20,10.93,10.84,10.10,239788.58,0.51,4.94,10.224,10.459,10.283,191163.52,203798.79,187805.70,2.40,600223,鲁商发展,区域地产,21.44,10.226333
2020-10-28,2020-10-28,10.00,10.41,10.33,10.00,191285.25,0.30,2.99,10.156,10.399,10.252,166084.56,196751.11,181417.41,1.91,600223,鲁商发展,区域地产,21.44,10.196667
2020-10-27,2020-10-27,9.93,10.27,10.03,9.85,197082.00,0.23,2.35,10.230,10.421,10.241,161132.00,206371.21,177640.29,1.97,600223,鲁商发展,区域地产,21.44,10.194333


In [211]:
%time c = df2[['ma5','ma10','ma20','ma30']].rolling(15,min_periods=1).apply(get_slope)

Wall time: 195 ms


c:\users\wuwukai\envs\h1\lib\site-packages\pandas\core\window\rolling.py:1415: RankWarning: Polyfit may be poorly conditioned
  return window_func(values, begin, end, min_periods)
c:\users\wuwukai\envs\h1\lib\site-packages\pandas\core\window\rolling.py:1415: RankWarning: Polyfit may be poorly conditioned
  return window_func(values, begin, end, min_periods)
c:\users\wuwukai\envs\h1\lib\site-packages\pandas\core\window\rolling.py:1415: RankWarning: Polyfit may be poorly conditioned
  return window_func(values, begin, end, min_periods)
c:\users\wuwukai\envs\h1\lib\site-packages\pandas\core\window\rolling.py:1415: RankWarning: Polyfit may be poorly conditioned
  return window_func(values, begin, end, min_periods)


In [230]:
(df2['close'] / df2['close'].rolling(15,min_periods=15).min() -1).round(2)

date
2019-10-08     NaN
2019-10-09     NaN
2019-10-10     NaN
2019-10-11     NaN
2019-10-14     NaN
2019-10-15     NaN
2019-10-16     NaN
2019-10-17     NaN
2019-10-18     NaN
2019-10-21     NaN
2019-10-22     NaN
2019-10-23     NaN
2019-10-24     NaN
2019-10-25     NaN
2019-10-28    1.09
2019-10-29    0.87
2019-10-30    0.73
2019-10-31    0.55
2019-11-01    0.59
2019-11-04    0.59
2019-11-05    0.54
2019-11-06    0.56
2019-11-07    0.32
2019-11-08    0.20
2019-11-11    0.00
2019-11-12    0.00
2019-11-13    0.03
2019-11-14    0.02
2019-11-15    0.05
2019-11-18    0.10
2019-11-19    0.09
2019-11-20    0.04
2019-11-21    0.14
2019-11-22    0.18
2019-11-25    0.06
2019-11-26    0.04
2019-11-27    0.02
2019-11-28    0.03
2019-11-29    0.00
2019-12-02    0.00
2019-12-03    0.01
2019-12-04    0.03
2019-12-05    0.03
2019-12-06    0.06
2019-12-09    0.10
2019-12-10    0.09
2019-12-11    0.13
2019-12-12    0.10
2019-12-13    0.10
2019-12-16    0.12
2019-12-17    0.13
2019-12-18    0.19
2019-12

In [250]:
from scipy.signal import find_peaks
import numpy as np

In [256]:
find_peaks(np.array([0,3,6,7,2,1,2,4,2,1,2,3]), distance=5)

(array([3], dtype=int64), {})

In [249]:
argrelextrema(np.array([0,3,6,7,2,1,2]),np.greater_equal,order=2)

(array([3, 6], dtype=int64),)

In [227]:
df3 = df2.sort_values(['t_date'],ascending=False)

In [287]:
pd.read_pickle(r"D:\work_dir\stock\src\stockProject\origin.pkl")

,jx_date,ma5,ma10,ma20,ma30,jx_counts,jx_names,symbol,t_date,open_price,high,close_price,low,volume,price_change,p_change,v_ma5,v_ma10,v_ma20,turnover,stock_name,industry,pe,bf_close,af_close,var,sw_class
0,2019-10-08,13.97,14.23,14.44,13.65,4,"13.97,14.23,14.44,13.65",002624,2019-10-08,27.70,28.40,27.31,27.19,79186.81,-0.39,-1.41,80421.45,87702.52,102259.38,0.68,完美世界,影视音像,23.18,NaN,0.01,0.018535,get_ma_A_ma30
1,2019-10-09,-0.22,-0.18,-0.02,0.09,1,"-0.22,-0.18,-0.02,0.09",002624,2019-10-09,27.32,27.59,27.48,27.09,43030.33,0.17,0.62,68089.77,86640.98,98644.09,0.37,完美世界,影视音像,23.18,NaN,-0.01,0.017967,get_ma_A_ma30
2,2019-10-10,-0.06,-0.13,-0.02,0.18,1,"-0.06,-0.13,-0.02,0.18",002624,2019-10-10,27.49,28.35,28.20,27.39,66474.72,0.72,2.62,62307.56,84422.04,96064.16,0.57,完美世界,影视音像,23.18,NaN,-0.04,0.013826,get_ma_A_ma30
3,2019-10-11,-0.03,-0.09,-0.02,0.19,1,"-0.03,-0.09,-0.02,0.19",002624,2019-10-11,28.20,28.73,28.47,27.72,79455.28,0.27,0.96,63393.58,82166.40,93439.39,0.68,完美世界,影视音像,23.18,NaN,-0.05,0.011641,get_ma_A_ma30
4,2019-10-14,0.03,-0.06,-0.01,0.19,2,"0.03,-0.06,-0.01,0.19",002624,2019-10-14,28.63,29.02,28.79,28.28,76804.53,0.32,1.12,68990.33,80051.55,92833.98,0.66,完美世界,影视音像,23.18,NaN,-0.05,0.009726,get_ma_A_ma30
5,2019-10-15,0.10,-0.05,-0.04,0.15,2,"0.1,-0.05,-0.04,0.15",002624,2019-10-15,28.40,28.45,27.51,27.48,84903.74,-1.28,-4.45,70133.72,75277.58,85770.14,0.73,完美世界,影视音像,23.18,NaN,0.03,0.008639,get_ma_A_ma30
6,2019-10-16,0.07,-0.08,-0.08,0.05,2,"0.07,-0.08,-0.08,0.05",002624,2019-10-16,27.60,27.88,27.35,27.13,68896.02,-0.16,-0.58,75306.86,71698.32,81920.22,0.59,完美世界,影视音像,23.18,NaN,0.05,0.007806,get_ma_A_ma30
7,2019-10-17,0.02,-0.09,-0.11,-0.03,1,"0.02,-0.09,-0.11,-0.03",002624,2019-10-17,27.28,27.85,27.51,27.22,53606.48,0.16,0.58,72733.21,67520.39,79601.59,0.46,完美世界,影视音像,23.18,0.01,0.05,0.007334,get_ma_A_ma30
8,2019-10-18,-0.09,-0.10,-0.13,-0.07,0,"-0.09,-0.1,-0.13,-0.07",002624,2019-10-18,27.51,27.84,27.34,26.96,54071.01,-0.17,-0.62,67656.36,65524.97,77548.53,0.47,完美世界,影视音像,23.18,-0.01,0.04,0.007565,get_ma_A_ma30
9,2019-10-21,-0.18,-0.08,-0.12,-0.06,0,"-0.18,-0.08,-0.12,-0.06",002624,2019-10-21,27.41,27.46,27.05,26.71,46105.46,-0.29,-1.06,61516.54,65253.44,75768.37,0.40,完美世界,影视音像,23.18,-0.04,0.07,0.008050,get_ma_A_ma30


In [259]:
(df3['close'].rolling(15,min_periods=1).max() / df3['close']   -1).round(2)

date
2020-11-06    0.00
2020-11-05    0.00
2020-11-04    0.06
2020-11-03    0.13
2020-11-02    0.16
2020-10-30    0.17
2020-10-29    0.13
2020-10-28    0.18
2020-10-27    0.22
2020-10-26    0.24
2020-10-23    0.21
2020-10-22    0.16
2020-10-21    0.14
2020-10-20    0.14
2020-10-19    0.13
2020-10-16    0.12
2020-10-15    0.13
2020-10-14    0.03
2020-10-13    0.04
2020-10-12    0.04
2020-10-09    0.06
2020-09-30    0.12
2020-09-29    0.11
2020-09-28    0.11
2020-09-25    0.09
2020-09-24    0.09
2020-09-23    0.06
2020-09-22    0.07
2020-09-21    0.05
2020-09-18    0.04
2020-09-17    0.05
2020-09-16    0.06
2020-09-15    0.03
2020-09-14    0.05
2020-09-11    0.03
2020-09-10    0.08
2020-09-09    0.05
2020-09-08    0.02
2020-09-07    0.01
2020-09-04    0.00
2020-09-03    0.00
2020-09-02    0.00
2020-09-01    0.06
2020-08-31    0.05
2020-08-28    0.08
2020-08-27    0.05
2020-08-26    0.00
2020-08-25    0.00
2020-08-24    0.00
2020-08-21    0.04
2020-08-20    0.06
2020-08-19    0.03
2020-08

In [237]:
(c>0).sum(axis=1)

date
2019-10-08    4
2019-10-09    2
2019-10-10    3
2019-10-11    4
2019-10-14    3
             ..
2020-11-02    3
2020-11-03    2
2020-11-04    2
2020-11-05    2
2020-11-06    2
Length: 266, dtype: int64